# Modelado y entrenamiento de Redes Neuronales Profundas

Objetivos:
    - Separar los dataset para entrenar distintos modelos:
        - Dataset completo.
        - Por sexo.
    - Modelar redes neuronales que tengan como objetivo predecir el comportamiento del animal a partir de los datos de:
        - 1 segundo de mediciones (capa de entrada de: 10 muestras * 4 Variables por muestra = 40 unidades de entrada).
        - 5 segundos de mediciones (capa de entrada de: 50 muestras * 4 Variables por muestra = 200 unidades de entrada). 
            - Predice el comportamiento del ultimo segundo.
        - Los comportamientos a predecir de acuerdo a los dataset separados son:
            - Dataset 1 - estudio3.csv - 12
            - Dataset 2 - tabla-resumen.csv - 10
    - Evaluar los modelos para cada tipo de dataset tomado.


Después del análisis exploratorio, pasamos a enumerar los pasos para entrenar las distintos modelos de redes neuronales.

1. Abrir datasets a considerar
2. Generar split de datos: Train, test, validation
3. Correr entrenamiento de los modelos y guardar los distintos modelos
4. Evaluar modelos sobre el mismo set de validación, que no fue utilizado en el proceso de entrenamiento de ninguno de los modelos.


## 1 - Carga de datasets
Se cargan los datasets previamente generados en objetos. 

In [16]:
import pandas as pd

estudio3_df = pd.read_csv("../data/1-intermedia/estudio3_dataset_per_second.csv")
resumen_df = pd.read_csv("../data/1-intermedia/tabla-resumen_dataset_per_second.csv")

ImportError: cannot import name 'dataframe_from_csv' from 'dataset_manipulation' (/home/jbaez/Documents/utn/gintea/rnr-meleros/notebooks/../src/dataset_manipulation/__init__.py)

## 2 - Generación split de datos
